In [ ]:
from __future__ import division
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from caspyr.utils import H5Reader
from caspyr.plotting import volshow
from argopt import DictAttrWrap
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
def imshow(im, cmap="Greys", origin="lower", title=None):
    ax = plt.imshow(im, cmap=cmap, origin=origin)
    plt.axis("off")
    if title:
        plt.title(title)

In [ ]:
dat = H5Reader("brainweb.raws/subject_04_sigma1_noise0.75_mMR.mat", prefix="MultiMaps_Ref/")
dat.PET = H5Reader("brainweb.raws/subject_04_sigma1_noise1_mMR.mat", prefix="MultiMaps_Ref/").PET[:]
d = dict()
for i in dat.keys():
    d[i] = getattr(dat, i)[:]
del dat
d = DictAttrWrap(d)

In [ ]:
ROI = slice(0, None), slice(100, -100), slice(100, -100)
volshow([(i, getattr(d, i)[ROI]) for i in d.d.keys()]);

In [ ]:
nReals = 5
PAD = 4
ROI = (19, 108), (128, 100+115), (120, 100+124)
ROI = [slice(i - PAD, j + PAD) for i, j in ROI]

In [ ]:
def load(nPats, nReals=5, counts=None, ROI=None, **kwargs):
    """
    Intended for loading training data, e.g.:
    >>> x, y = load(10, 1, counts=[30, 300])

    @param counts  : [low, high] in millions [default: [3, 300]]
    @param ROI  : ZXY (even though return is ZYX)
    @return ndarray, shape (len(counts), nPats*nReals, D, H, W, Ch)
      Ch is 3=<T1, PSF, PET> if `counts > 0`, 1=<TRUTH> otherwise.
    """
    from caspyr.utils import Globber, H5Reader
    from os import path
    import numpy as np
    import logging
    from tqdm.auto import tqdm
    glob = Globber.glob
    log = logging.getLogger(__name__)

    DAT_ROOT = "/data/cc16/apirl/o"
    #"brainweb_PETpsf_1_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3_{:03d}.mat"
    subjects = sorted({i[47:49] for i in glob(path.join(
        DAT_ROOT, "reconAPIRL_brainweb_subject_*-S_1-NP_1-NT1_0.75-C_*_t3.mat"))})
    log.debug("subj:" + ', '.join(subjects))

    counts = counts or (3.01, 301)
    ROI = ROI or (slice(0, None),) * 3
    assert len(ROI) == 3
    ROI = tuple(ROI)

    res = [] # [counts, nReals + nReals_psf + T1 + PET, z,y,x]
    for c in tqdm(counts, desc="counts"):
        vols = []
        for subj in tqdm(subjects[:nPats], desc="subject", leave=False):
            d = path.join(DAT_ROOT,
                "reconAPIRL_brainweb_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3.mat".format(
                    subj, abs(c)*1e6))
            d = H5Reader(d, prefix="reconAPIRL")
            if c < 0:
                i = d.PET[ROI][None, :,:,:, None] * d.scale_factor[0,0]  # 1zyx1
                i = np.repeat(i, min(d.mlem.shape[3], nReals), axis=-1)  # 1zyxN
            else:
                i = np.concatenate((
                    d.mlem_psf[ROI + (slice(0, nReals),)][None],
                    d.mlem[ROI + (slice(0, nReals),)][None],
                ))  # 2zyxN
                i = np.concatenate((np.repeat(d.T1[ROI][None, :,:,:, None], i.shape[-1], axis=-1), i))  # 3zyxN
                #d.PET[ROI][None, :,:,:,None] * d.scale_factor[0,0]
            i = i.transpose((4, 1,3,2, 0))  # Nzxy3
            vols.append(i)
        res.append(np.concatenate(vols))
    return res

# low->high
#dat = load(10, nReals=nReals, counts=[3.01, 301], ROI=ROI) # 2, nReals*10, z, y, x, 3
#np.savez_compressed('bweb-3.01-301.npz', x=dat[0], y=dat[1])

dat = load(10, nReals=nReals, counts=[3.01, -3.01], ROI=ROI)
np.savez_compressed('bweb-3.01-T.npz', x=dat[0], y=dat[1])

#dat = load(10, nReals=nReals, counts=[30.1, -30.1], ROI=ROI)
#np.savez_compressed('bweb-30.1-T.npz', x=dat[0], y=dat[1])

In [ ]:
# bias-var
from argopt import DictAttrWrap
from viper.utils import stats

IDX = DictAttrWrap(dict(TRUE=0, T1=0, PSF=1, PET=2))

tru = dat[1][:, :,:,:, IDX.TRUE]
pet = dat[0][:, :,:,:, IDX.PET]
psf = dat[0][:, :,:,:, IDX.PSF]

#volshow(np.concatenate((tru[:4], psf[:4], pet[:4])));

In [ ]:
for i in range(nReals):
    reals = slice(i*nReals, (i + 1)*nReals)
    #volshow(tru[reals])

    """
    scale = 1 / (tru[reals][0] ** 2).mean()
    bias = (((pet[reals].mean(axis=0) - tru[reals][0]) ** 2).mean() * scale) ** 0.5
    stdv = ((pet[reals].var(axis=0, ddof=1)).mean() * scale) ** 0.5
    # NOTE: use ddof=0 above to make nrmse = hypot(bias, stdv)
    #nrmse = (((pet[reals] - tru[reals][0]) ** 2).mean() * scale) ** 0.5
    """
    bias, stdv = stats.biasStdMask(pet[reals], tru[reals][:1])
    print("pet", bias, stdv, np.hypot(bias, stdv))

    bias, stdv = stats.biasStdMask(psf[reals], tru[reals][:1])
    print("psf", bias, stdv, np.hypot(bias, stdv))

In [ ]:
def biasStdv(nPats, nReals=5, counts=3.01, ROI=None, psf=False, **kwargs):
    """
    >>> biasStdv(10, 1, counts=[30, 300])

    @param counts  : in millions
    @param ROI  : ZXY (even though return is ZYX)
    @return ndarray, shape ((MLEM, PS), (bias, stdv), nItr)
    """
    from caspyr.utils import Globber, H5Reader
    from viper.utils import stats
    from viper.constants import SIGMA2FWHM_MMRzyx
    from viper.imsample import gauss
    from os import path
    import numpy as np
    import logging
    from tqdm.auto import tqdm, trange

    glob = Globber.glob
    log = logging.getLogger(__name__)

    DAT_ROOT = "/data/cc16/apirl/o"
    #"brainweb_PETpsf_1_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3_{:03d}.mat"
    subjects = sorted({i[47:49] for i in glob(path.join(
        DAT_ROOT, "reconAPIRL_brainweb_subject_*-S_1-NP_1-NT1_0.75-C_*_t3.mat"))})
    log.debug("subj:" + ', '.join(subjects))

    c = counts * 1e6
    ROI = ROI or (slice(0, None),) * 3
    assert len(ROI) == 3
    ROI = tuple(ROI)

    res = []  # [nPat, [MLEM, PostSmooth], nItr, [bias, stdv]]
    for subj in tqdm(subjects[:nPats], unit="subject", leave=False):
        tru = path.join(DAT_ROOT,
                "reconAPIRL_brainweb_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3.mat".format(
                    subj, c))
        tru = H5Reader(tru, prefix="reconAPIRL")
        tru = tru.PET[ROI][None] * tru.scale_factor[0,0]  # 1zxy

        iters = glob(path.join(DAT_ROOT,
            "brainweb_PET_0_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3_*.mat".format(
                subj, c)))
        iters = [i[-7:-4] for i in iters]
        log.debug("itr:" + ' '.join(iters))

        bs = []
        for i in tqdm(iters, leave=False):
            reals = glob(path.join(DAT_ROOT,
                "brainweb_PET{}_*_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3_{}.mat".format(
                "psf" if psf else "", subj, c, i)))
            #log.debug(reals)
            reals = [H5Reader(d).Img[ROI] for d in reals]  # Nzxy
            #bs.append(stats.biasStd(np.array(reals), tru))
            bs.append(stats.biasStdMask(np.array(reals), tru))
        #res.append(bs)

        last = np.array(reals)
        bsPS = []
        for i in tqdm(np.linspace(0, 25, num=len(iters)), unit_scale=25.0/len(iters),
                      unit="mm", desc="PS", leave=False):
            #bsPS.append(stats.biasStd(np.array(reals), tru))
            sigma = [i / s for s in SIGMA2FWHM_MMRzyx]
            bsPS.append(stats.biasStdMask(np.array([gauss(i, sigma) for i in last]), tru))
        res.append((bs, bsPS))
    return np.mean(res, axis=0).transpose((0, 2, 1))
# low
logging.getLogger().setLevel(logging.INFO)
bsPet = biasStdv(1, nReals=nReals, counts=3.01, psf=False, ROI=ROI)
bsPsf = biasStdv(1, nReals=nReals, counts=3.01, psf=True, ROI=ROI)
# high
#bias, var = biasStdv(10, nReals=nReals, counts=3.01, ROI=ROI)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(bsPet[0, 1], bsPet[0, 0], 'o-', ms=4, label="MLEM")
ax.plot(bsPsf[0, 1], bsPsf[0, 0], 'o-', ms=4, label="MLEM+RM")

i = np.hypot(bsPet[1, 1], bsPet[1, 0]); i = np.where(i == i.min())[0]
ax.plot(bsPet[1, 1], bsPet[1, 0], 'ko-', markevery=i, label="PS up to 25mm")

i = np.hypot(bsPsf[1, 1], bsPsf[1, 0]); i = np.where(i == i.min())[0]
ax.plot(bsPsf[1, 1], bsPsf[1, 0], 'ko-', markevery=i)

plt.xlabel(r"Standard deviation, $\sigma$/[%]")
plt.ylabel(r"Bias, $b$/[%]")
plt.xlim(0, None)
plt.legend();